In [1]:
import os


In [2]:
%pwd


'd:\\Inventory-Management-and-Predictive-Analytics-with-IOT\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'd:\\Inventory-Management-and-Predictive-Analytics-with-IOT'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    n_estimators: int
    min_weight_fraction_leaf: float
    min_samples_split: int
    min_samples_leaf: int
    min_impurity_decrease: float
    max_leaf_nodes: int
    max_depth: int

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import *
from sklearn.ensemble import RandomForestRegressor


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema.name,
            n_estimators = params.n_estimators,
            min_weight_fraction_leaf = params.min_weight_fraction_leaf,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            min_impurity_decrease = params.min_impurity_decrease,
            max_leaf_nodes = params.max_leaf_nodes,
            max_depth = params.max_depth
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject import logger
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self,x_train,y_train):
        try:
            rf_reg = RandomForestRegressor(n_estimators = self.config.n_estimators,
                                            min_weight_fraction_leaf = self.config.min_weight_fraction_leaf,
                                            min_samples_split = self.config.min_samples_split,
                                            min_samples_leaf = self.config.min_samples_leaf,
                                            min_impurity_decrease = self.config.min_impurity_decrease,
                                            max_leaf_nodes = self.config.max_leaf_nodes,
                                            max_depth = self.config.max_depth)
                                                    
            rf_reg.fit(x_train,y_train)
            return rf_reg
        except Exception as e:
            raise e

    
    def train(self):
        train_arr = load_numpy_array_data(self.config.train_data_path)
        test_arr = load_numpy_array_data(self.config.test_data_path)


        x_train, y_train, x_test, y_test = (
                train_arr[:, :-1],
                train_arr[:, -1],
                test_arr[:, :-1],
                test_arr[:, -1],
            )
        
        model = self.train_model(x_train, y_train)


        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-11-21 16:36:22,692: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-21 16:36:22,703: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-21 16:36:22,707: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-21 16:36:22,710: INFO: common: created directory at: artifacts]
[2023-11-21 16:36:22,712: INFO: common: created directory at: artifacts/model_trainer]
